<a href="https://colab.research.google.com/github/e0zturk/colabfolding_turkish/blob/main/folding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Google Colab**
## ColabFOLD & AlphaFold

[Resmi Repo: LocalColabFold](https://github.com/YoshitakaMo/localcolabfold) | [Yöntem: AlphaFold2](https://nature.com/articles/s41586-021-03819-2)

Bu çalışma defteri, **LocalColabFold** mimarisini kullanarak amino asit dizilerinden (Fasta) yüksek doğrulukta **3 boyutlu protein yapıları (.pdb)** ve kompleksleri oluşturmak için tasarlanmıştır.

**LocalColabFold Nedir?**
Google DeepMind tarafından geliştirilen **AlphaFold2** algoritmasının, daha hızlı ve erişilebilir bir versiyonudur. **MMseqs2** algoritması kullanılarak oluşturulan derin Çoklu Dizi Hizalaması (MSA) ve PDB veritabanındaki homolog şablonların (templates) entegrasyonuna dayanır. Bu yöntem, pLDDT skorunu maksimize ederek deneysel verilere en yakın topolojiyi sunar.

**Özellikler:**
* **Tahmin algoritması**: **MMseqs2**

* **Hız:** Notebook editörü, T4 GPU üzerinde çalışacak şekilde, 3 cycle optimizasyon döngüsünde (varsayılan); 100 amino asitlik ortalama bir sekansın tam analizini (MSA + 5 Model + Relaxation) yaklaşık 10 dakika sürdüğünü yaptığı deneme testlerinde doğrulamıştır. Hızınızı arttırmak için COLAB aboneliği & birim satın alma gerçekleştirebilirsiniz.

* **Kompleks Modelleme:** Varsayılan olarak AlphaFold2_multimer_v3 motoru kullanılır. Bu sürüm, protein-protein etkileşim arayüzlerini (interface) çözmek için özel olarak eğitilmiştir; dimer, trimer veya daha büyük oligomerik yapıların stokiyometrik analizini mümkün kılar.

### ⭕SİSTEM GEREKSİNİMLERİ :
* 17+ GB DRİVE DEPOLAMA ALANI

---
Colabfold programının dosya paketleri yaklaşık 16.3 GB yer kaplamaktadır. Giriş yapılan Google Drive hesabınızın yeterli depolama alanı olup olmadığını kontrol ediniz. Aksi takdirde kurulum gerçekleşmez.

## **Hazırlık ve Filtrasyon**


Çalışmanın sağlıklı yürütülmesi adına Google Colab sizden; **verilerinizi** tanımak, işlemek, çalıştırmak ve sonuçları kaydetmek amacıyla `Google Drive` hesabına erişim izni isteyecektir. **Lütfen erişim izinlerini onaylayınız.**

**İlk defa çalıştırmanız halinde Drive dizinine otomatik `Folding/data` dosyaları tanımlanacaktır. İlk çalıştırılmadan alınan HATA'yı tekrar edilmemesi adına çalışma dosyasını `raw_data` .fasta formatında yüklemeniz gerekmektedir.**

**`data/` klasörüne yüklediğiniz `.fasta` dosyasının tam adını hücrede girdi olarak verilmesi elzemdir.**

**`raw_data = "xxxx.fasta"`**


`raw_data` olarak atadığınız değişken dosyası içerisinde bulunan bazı bozuk dosyalar ve tekrar eden diziler filtreleme aşamasından geçer.

---
### BİLGİ ℹ️
- Hücreyi çalıştırdıktan sonra filtrelenmiş (`.fasta`) dosyasının ne kadar sağlam sekans içerdiği bilgisi çıktıda `Sağlam sekans sayısı:` şeklinde paylaşılmıştır. Bir sonraki bölümde kullanmak üzere not etmeyi **UNUTMAYINIZ**
* Hücre, **`raw_data`** önceden yüklenmiş filtresiz dosyasını (`xxx.fasta`) not defteri çalıştırıldığında `MyDrive/Folding` klasöründen silecektir. Onun yerine filtrelenmiş, sağlam yapılar içeren aynı isimde (`xxx.fasta`) dosyası üretecektir. İşlenmemiş `.fasta` dosyanızı local (kişisel bilgisayarınız) dosyalarda yedeklediğinizden **emin** olun.

In [2]:
import os
from google.colab import drive

#___________ÇALIŞMA ALANI__________
raw_data = "HPV.fasta"  # örnek dosya (kendiniz yüklemelisiniz, değişkene atamalısınız)
Work_on_dir = "/content/drive/MyDrive/Folding"
#__________________________________

print("🔌 Google Drive bağlanıyor...")
try:
    if not os.path.exists('/content/drive'):
        drive.mount('/content/drive')
except Exception as e:
    print("❌ Drive bağlantısı kurulamadı. Lütfen sol taraftaki 'Dosyalar' simgesinden Drive simgesine tıklayarak manuel bağlamayı deneyin.")
    raise e

data_dir = os.path.join(Work_on_dir, "data")
folders_created = False

if not os.path.exists(Work_on_dir):
    os.makedirs(Work_on_dir, exist_ok=True)
    folders_created = True

if not os.path.exists(data_dir):
    os.makedirs(data_dir, exist_ok=True)
    folders_created = True

if folders_created:
    print(f"ℹ️ '{Work_on_dir}' ve 'data/' klasörleriniz otomatik olarak oluşturulmuştur.")

file_path = os.path.join(data_dir, raw_data)

if not os.path.exists(file_path):
    print(f"\n⚠️ Lütfen data klasörüne ({data_dir}) çalışmak istediğiniz '{raw_data}' dosyasını yükleyiniz.")
else:
    print(f"\n✔️ Bağlanma başarılı. İşlenecek dosya doğrulandı: {file_path}\n")

    seen_sequences = set()
    clean_sequences = []
    removed_x_count = 0
    removed_dup_count = 0

    current_header = None
    current_seq_lines = []

    def process_sequence(header, seq_lines):
        global removed_x_count, removed_dup_count
        full_seq = "".join(seq_lines).strip().upper()
        if not full_seq: return
        if full_seq in seen_sequences:
            removed_dup_count += 1
            return
        if full_seq.count("X") > 3:
            removed_x_count += 1
            return
        seen_sequences.add(full_seq)
        clean_sequences.append((header, full_seq))

    with open(file_path, 'r') as f:
        for line in f:
            line = line.strip()
            if not line: continue
            if line.startswith(">"):
                if current_header:
                    process_sequence(current_header, current_seq_lines)
                current_header = line
                current_seq_lines = []
            else:
                current_seq_lines.append(line)
        if current_header:
            process_sequence(current_header, current_seq_lines)

    total_clean = len(clean_sequences)
    if total_clean > 0:
        with open(file_path, 'w') as f:
            for header, seq in clean_sequences:
                f.write(f"{header}\n{seq}\n")

        total_raw = total_clean + removed_dup_count + removed_x_count
        average = sum(len(seq) for _, seq in clean_sequences) / total_clean

        print(f"📊 İstatistikler:")
        print(f"   - Ham Veri       : {total_raw}")
        print(f"   - Tekrar Eden    : {removed_dup_count}")
        print(f"   - Bozuk (X > 3)  : {removed_x_count}")
        print(f"   - Eliminasyon %  : %{((1-(total_clean/total_raw))*100):.2f}")
        print(f"\n✔️ Filtreleme başarıyla gerçekleşti.")
        print(f"Sağlam sekans sayısı: {total_clean}")
        print(f"Ortalama Sekans Uzunluğu: {average:.2f}")
    else:
        print("⚠️ HATA: Filtreleme sonucu elde kalan veri yok.")

🔌 Google Drive bağlanıyor...
Mounted at /content/drive

✔️ Bağlanma başarılı. İşlenecek dosya doğrulandı: /content/drive/MyDrive/Folding/data/HPV.fasta

📊 İstatistikler:
   - Ham Veri       : 121
   - Tekrar Eden    : 0
   - Bozuk (X > 3)  : 0
   - Eliminasyon %  : %0.00

✔️ Filtreleme başarıyla gerçekleşti.
Sağlam sekans sayısı: 121
Ortalama Sekans Uzunluğu: 306.34


## **Veri işleme**
**İşlenmiş veriler üzerinde gerekli tüm ayarlamaların gerçekleştiği kısımdır.**
###**SİSTEM AYARLARI:**
1. parçalama stratejisi ayarları
-  mode : `aim` ya da `rate`
- parça sayısı: `split_param` (integer)
2. RUN (çalıştırma) ayarları
- Çalıştırma modu : `monomer` ya da `multimerv3`
- Çalıştırılmak istenen parça sayısı : `kota` (integer)
- Kaldığınız yerden devam etme: `take_up` = (boolen) , `resume_as` = (integer)
- Sunucunun optimizasyon döngüsü: `cycle`(integer)

---
#####NOTLAR:
* İşlenmiş dosyanızın kaç tane sekans içerdiği bilgisini `Hazırlık` bölümüne giderek öğrenebilirsiniz.

* Fasta formatındaki dosyanızın sekans sayısının uzun olması programın tamamlanma süresini uzatabilir. Bu süreyi en aza indirmek için `.fasta` dosyanız parçalara ayırmanız önemlidir.

* Parçalanmış dosyalar _`Folding/*fasta_parts`_ dizinine kaydedilir.

* Yeni bir işleme başlanıldığında önceki çalışmalarda oluşturulmuş _`*_fasta-parts`_ klasörü **silinmektedir**.

---
##### Bölme stratejisi
* `rate` : Parçalamak için orantısal metod kullanır. Ortalama olarak istenilen `split_param` parça sayısı kadar parçaya bölünür. Örneğin "rate", 5 olarak değişken ataması yapmak '**Toplam 5 parça olacak şekilde böl**' anlamına gelir.

* `aim`: Parçaların hedef sekans sayısına göre dosyanızı böler. `split_param` olarak yazdığınız uzunlukta eşit parçalar üretir. Son parça eşit dağılan sekans parçalarından eksik kalabilir. Örneğin `"aim"` , `30` ataması yapmak '**Parçalar 30 sekans içerecek şekilde böl**' anlamına gelir.

---
##### RUN (çalıştırma) stratejisi
* Çalıştırma modunu dimer, trimer yapıları için varsayılan `multimerv3` da tutmanızı öneriyoruz.
* `kota`: Parçalanan `.fasta` dosyalarını  RUN (çalıştırma) sunucusuna sırasıyla döngü halinde çalıştırmanızı sağlar. Tavsiye edilen `1`'dir.
* `take_up` kaldığınız yerden devam ettirmek için `True` (açık) boolen değişkeni, varsayılan olarak kapatmak için `False` boolen kullanabilirsiniz. `resume_as` hangi parçadan devam etmek istediğinizi belirtmek içindir. (örn: `xxx_part3.fasta`) -> `resume_as = 3`
* `cycle` değişkeni colabfold'un `.pdb` dosya taramasında yaptığı optimizasyon döngü sayısıdır. Programın hızını etkileyebilir. Hız sıralaması : 1,3,5,6,10,12. Tavsiye edilen dengeli tarama '3' olarak ayarlanmıştır.


---
#####⚠️UYARI : Çalıştırmadan önce `--- KULLANICI AYARLARI ---` kısmını kendi çalışmanıza göre **revize** ediniz.

In [3]:
import os
import math
import shutil

# --- KULLANICI AYARLARI (SİSTEM AYARLARI) ---
mode = "aim"  #rate
split_param = 1
run_mode = "monomerv2" #monomerv2
kota = 1  # oluşan parçaları {kota} kere döngüyle çalıştırır.
take_up = False #False
resume_as = 6  #part{i} ve sonrası
cycle = 1 # hız sıralaması: 1,3,5,6,10,12
# --------------------------------------------

try:
    if not os.path.exists(file_path):
        raise FileNotFoundError
    print(f"Sistem Hazır. İşlenecek Dosya: {os.path.basename(file_path)}\n")
    print(f"\n⚙️Ayarlar: Mod={mode}, Param={split_param}, RunMod={run_mode}, Kota={kota}\n\
                Cycling={cycle},TakeUp={take_up}, ResumeAs={resume_as}\n")
except (NameError, FileNotFoundError):
    print("❌ HATA: 'Hazırlık ve Filtrasyon' bölümü çalıştırılmamış veya dosya bulunamadı.")
    print("Lütfen önce Hazırlık bölümünü çalıştırın.")
else:
    # --- TEMİZLİK ---
    print("🧹'*_fasta_parts' klasörleri taranıyor...")
    for item in os.listdir(Work_on_dir):
        item_path = os.path.join(Work_on_dir, item)
        if os.path.isdir(item_path) and item.endswith("_fasta_parts"):
            try:
                shutil.rmtree(item_path)
                print(f"\nSilme işlemi başarılı. yeniden boş dizin oluşturuluyor.")
            except Exception as e:
                print(f"⚠️ Silinemedi: {item} -> {e}")
    # ----------------

    abs_path = os.path.abspath(file_path)
    base_name = os.path.splitext(os.path.basename(abs_path))[0]
    output_dir = os.path.join(Work_on_dir, f"{base_name}_fasta_parts")

    os.makedirs(output_dir, exist_ok=True)
    # ------------------------

    clean_sequences = []
    current_header = None
    current_seq_lines = []

    with open(abs_path, 'r') as f:
        for line in f:
            line = line.strip()
            if not line: continue
            if line.startswith(">"):
                if current_header:
                    full_seq = "".join(current_seq_lines).strip()
                    if full_seq: clean_sequences.append((current_header, full_seq))
                current_header = line
                current_seq_lines = []
            else:
                current_seq_lines.append(line)
        if current_header:
            full_seq = "".join(current_seq_lines).strip()
            if full_seq: clean_sequences.append((current_header, full_seq))

    total_seqs = len(clean_sequences)
    chunks = []

    if total_seqs > 0:
        if mode == "rate":
            if split_param > 0:
                chunk_size = math.ceil(total_seqs / split_param)
                for i in range(0, total_seqs, chunk_size):
                    chunks.append(clean_sequences[i:i + chunk_size])
                print(f"\nℹ️ Mod: RATE -> Toplam {total_seqs} sekans, ortalama {len(chunks)} dosyaya bölünüyor.\n")

        elif mode == "aim":
            if split_param > 0:
                for i in range(0, total_seqs, split_param):
                    chunks.append(clean_sequences[i:i + split_param])
                print(f"\nℹ️ Mod: AIM -> Her dosyada en fazla {split_param} sekans olacak.\n")

        for i, chunk in enumerate(chunks):
            output_file = os.path.join(output_dir, f"{base_name}_part{i + 1}.fasta")
            with open(output_file, 'w') as out_f:
                for header, seq in chunk:
                    out_f.write(f"{header}\n{seq}\n")

        print(f"\n✅ İŞLEM TAMAM: Toplam {len(chunks)} parça oluşturuldu.")

    else:
        print("⚠️ HATA: Dosya boş veya okunamadı. Lütfen Hazırlık bölümünü kontrol edin.")

Sistem Hazır. İşlenecek Dosya: HPV.fasta


⚙️Ayarlar: Mod=aim, Param=1, RunMod=monomerv2, Kota=1
                Cycling=1,TakeUp=False, ResumeAs=6

🧹'*_fasta_parts' klasörleri taranıyor...

Silme işlemi başarılı. yeniden boş dizin oluşturuluyor.

ℹ️ Mod: AIM -> Her dosyada en fazla 1 sekans olacak.


✅ İŞLEM TAMAM: Toplam 121 parça oluşturuldu.


##**LOCAL COLAB FOLD KONTROLÜ**
`localcolabfold` dosyalarının colab notebook'unun içinde varlığını sorgulamaktadır. Eğer yoksa `Drive` bağlantısı üzerinden `MyDrive` içinde `colabfold_yedek.tar` adında editörün sunduğu yedekleme dosyası kontrol edilir. Yedek kurulum sırasında işlem yarıda kesilir ise yukarıdaki menüden;
`Çalışma zamanı > Çalışma zamanı ve bağlantısını kes ve sil`
yaptıktan sonra tekrar Drive bağlanarak çalıştırın.

---
######Not
* `'LocalColabFold zaten yüklü'` ise lütfen $PATH aşamasına geçiniz.
* Aksi takdirde manuel kurulum başlatınız.


In [4]:
import os
from google.colab import drive

try:
    Work_on_dir
except NameError:
    Work_on_dir = "/content/drive/MyDrive/Folding"

install_path = "/content/localcolabfold"
backup_path = os.path.join(Work_on_dir, "colabfold_yedek.tar")

if os.path.exists(install_path):
    print("✅ DURUM: LocalColabFold zaten yüklü.\n")
    print("Manuel kurulumu atlayıp $PATH bölümüne geçebilirsiniz.")

else:
    print("Drive kontrol ediliyor...\n")
    if not os.path.exists('/content/drive'):
        drive.mount('/content/drive')

    if os.path.exists(backup_path):
        print(f"📥 Drive yedeği bulundu: {backup_path}\n")
        print("⏳ Yedek dosyası şimdi açılıyor (Hızınıza göre 5-10 dk sürebilir)...")

        !cp "{backup_path}" /content/
        !tar -xf /content/colabfold_yedek.tar -C /content/
        print("✅ Yedek .tar dosyası notebook'a yüklendi.")
    else:
        print(f"❌ '{backup_path}' yolunda yedek bulunamadı.")
        print("Lütfen bir sonraki hücreyi çalıştırarak 'Manuel Kurulum' yapınız.")

Drive kontrol ediliyor...

📥 Drive yedeği bulundu: /content/drive/MyDrive/Folding/colabfold_yedek.tar

⏳ Yedek dosyası şimdi açılıyor (Hızınıza göre 5-10 dk sürebilir)...
✅ Yedek .tar dosyası notebook'a yüklendi.


## **Manuel Kurulum - ColabFold**

Bu bölüm, protein modelleme için gerekli olan AlphaFold2 motorunu ve temel bağımlılıkları sisteme sunucudan kurar.

---
#####**Sistemin Çalışma Mantığı:**
**Girdi:**
* Amino asit dizilerini içeren metin dosyaları (.fasta).

**Çıktılar:**
* **_relaxed_...pdb:** Fiziksel enerji minimizasyonu (Amber) uygulanmış, atomik çakışmaların giderildiği nihai ve en kararlı model dosyası.
* **_unrelaxed_...pdb:** Algoritmanın ürettiği, henüz fiziksel optimizasyon yapılmamış ham model dosyası.
* **.json Dosyaları:** Modellerin güvenilirlik skorları ve sayısal metrik verileri.
* **.png Grafikleri:** Yapısal güvenilirlik (pLDDT) ve hizalama hatası (PAE) analiz görselleri

---
####⚠️Uyarı
* localcolabfold dosyaları kurulumu için lütfen işlemin tamamen bitmesini bekleyiniz.\
Bu işlem 10-15 dakika sürebilir
* Eğer ilk defa not defterini çalıştırıyorsanız, manuel kurulumundan sonra otomatize kurulum için dosyaların yedeği çıkarılacaktır. (`colabfold_yedek.tar`)\
Bu işlem de 10-20 dakika daha zaman alabilir.
---

In [5]:
import os
import shutil

install_path = "/content/localcolabfold"
backup_path = os.path.join(Work_on_dir, "colabfold_yedek.tar")

free_gb = shutil.disk_usage("/content").free / (1024**3)
if free_gb < 17:
    raise SystemExit(f"❌ Disk alanı yetersiz: {free_gb:.2f} GB")

force_reinstall = not os.path.exists(backup_path)

if force_reinstall or not os.path.exists(install_path):
    if os.path.exists(install_path):
        !rm -rf {install_path}

    print("🌍 MANUEL KURULUM başlatılıyor...")
    print("⚠️ Yaklaşık 17 GB indirme yapılacak. Bu işlem 10-15 dk sürebilir...\n")
    !curl -fsSL https://pixi.sh/install.sh | bash
    !git clone https://github.com/YoshitakaMo/localcolabfold.git {install_path}

    %cd {install_path}
    !/root/.pixi/bin/pixi install

    if os.path.exists(os.path.join(install_path, ".pixi")):
        print("⏳ Paketleme işlemi başlıyor...")
        %cd /content
        !tar -cf colabfold_yedek.tar localcolabfold
        os.makedirs(Work_on_dir, exist_ok=True)
        !mv colabfold_yedek.tar "{backup_path}"
        print("✅ İŞLEM TAMAM: Pixi ile kurulum ve yedekleme başarılı.")
    else:
        raise SystemExit("❌ HATA: Pixi ortamı oluşturulamadı!")
else:
    print("✅ Sistem zaten yüklü.")

✅ Sistem zaten yüklü.


##**PATH$**
---
Kurulan dosyaları sisteme tanıtır ve AlphaFold2 motorunun çalışıp çalışmadığını test eder.


In [6]:
import os

bin_path = "/content/localcolabfold/.pixi/envs/default/bin"

if os.path.exists(bin_path):
    if bin_path not in os.environ['PATH']:
        os.environ['PATH'] = f"{bin_path}:{os.environ['PATH']}"
        print(f"🔗 PATH ayarlandı: {bin_path}\n")

    os.environ['MPLBACKEND'] = 'Agg'

    print("Sistem Test Ediliyor...")
    !colabfold_batch --help | head -n 5

    print("\n✅ SİSTEM BİLEŞENLERİ VE Colabfold HAZIR.")
else:
    print("❌ KRİTİK HATA: Kurulum tamamlanmış görünmüyor.")
    raise SystemExit("PATH bulunamadığı için işlem durduruldu.")

🔗 PATH ayarlandı: /content/localcolabfold/.pixi/envs/default/bin

Sistem Test Ediliyor...
usage: colabfold_batch [-h] [--msa-only]
                       [--msa-mode {mmseqs2_uniref_env,mmseqs2_uniref_env_envpair,mmseqs2_uniref,single_sequence}]
                       [--pair-mode {unpaired,paired,unpaired_paired}]
                       [--pair-strategy {complete,greedy}] [--templates]
                       [--custom-template-path CUSTOM_TEMPLATE_PATH]

✅ SİSTEM BİLEŞENLERİ VE Colabfold HAZIR.


## Colab Fold **RUN** ▶️
Sistem Bileşenlerinin ve Fasta dosyalarınızın hazır olduğundan eminseniz `RUN` başlatabilirsiniz.

---
##### ℹ️ BİLGİLENDİRME:
* Colabfold, **`Folding/...fasta_parts`** klasörü içindeki oluşturulan dosyaları çalıştırabilir.

* Çıktılar `colabfold_results` klasörüne kaydedilecektir.

* **Önceden** elde edilmiş eski sonuçlar **silinmektedir** ve yeniden temiz bir şekilde `colabfold_results` klasörüne  çıktılar yüklenecektir.

---
#####⚠️UYARI⚠️:
**Tüm notebook hücrelerinin sırasıyla yürütüldüğü senaryoda** sistem otomatik olarak sonuçları bilgisayarınızın `Downloads` klasörüne indirecektir. Diğer hücrelerin çalıştırılmasını önemsemeden ham çıktıyı almak istiyorsanız Drive dizininizdeki `Folding/colabfold_results` dosyasını (**hücreyi tekrar çalıştırmadan**) kontrol ediniz.

---

####Varsayılan ayarlar
- **`--templates`**:PDB veritabanındaki benzer yapıları "iskelet" olarak kullanır.
- **`--pair-mode unpaired_paired`**: Geniş tarama modu.
- **`--model-type alphafold2_multimer_v3`**: Dimer, trimer yapıları keşfeder
- **`--num-recycle 3`**: Düşük derecede optimizasyon döngüsü.

- **`--amber`** and **`use_gpu_relax`**: Yüksek doğrlulukta relaxed.pdb modellerini oluşturur. (`/colabfold_results` 'u inceleyin). Varsayılan olarak `False` durumdadır. Etkinleştirilmesi yüksek RAM gücü (VRAM) kullanmayı gerektirir. VRAM ve GPU birimleri için [Google Colab Ücretli hizmetler](https://colab.research.google.com/signup?utm_source=resource_tab&utm_medium=link&utm_campaign=payg_learn_more&authuser=1)'i ziyaret edebilirsiniz.



In [7]:
import os
import shutil
import json
import glob
import sys
import subprocess
import signal

use_amber = False  #True VRAM etkinleştirildiğinden emin iseniz 'True' yapılabilir.
use_gpu_relax = False  #True

try:
    Work_on_dir
except NameError:
    Work_on_dir = "/content/drive/MyDrive/Folding"

clean_target_dir = os.path.join(Work_on_dir, "colabfold_results")
if os.path.exists(clean_target_dir):
    try:
        shutil.rmtree(clean_target_dir)
        print(f"🧹 Temizlik Yapıldı: '{clean_target_dir}' ve içeriği silindi.")
    except Exception as e:
        print(f"⚠️ Temizlik sırasında hata: {e}")

if 'run_mode' in locals() and run_mode == "multimerv3":
    model_type_cli = "alphafold2_multimer_v3"
else:
    model_type_cli = "alphafold2_ptm"

try:
    CALISMA_YOLU = Work_on_dir
    base_name_auto = os.path.splitext(raw_data)[0]
    parts_dir = os.path.join(CALISMA_YOLU, f"{base_name_auto}_fasta_parts")
    output_dir = os.path.join(CALISMA_YOLU, "colabfold_results")

    found_files = glob.glob(os.path.join(parts_dir, f"{base_name_auto}_part*.fasta"))
    total_parts = len(found_files)

    if total_parts == 0:
        raise FileNotFoundError(f"Klasörde hiç fasta parçası bulunamadı: {parts_dir}")

    start_at = 1
    should_clean = True

    batch_quota = kota if 'kota' in locals() else total_parts

    if 'take_up' in locals() and take_up and 'resume_as' in locals():
        should_clean = False
        start_at = resume_as
        if start_at > total_parts:
            print(f"\n⛔ HATA: İstediğiniz başlangıç ({start_at}), toplam parçadan ({total_parts}) büyük!")

    end_at = start_at + batch_quota - 1
    if end_at > total_parts:
        end_at = total_parts

    if should_clean:
        if os.path.exists(output_dir):
            try:
                shutil.rmtree(output_dir)
                print("🧹 Temizlik: Eski sonuçlar silindi.")
            except: pass
        os.makedirs(output_dir, exist_ok=True)
    else:
        os.makedirs(output_dir, exist_ok=True)


    original_fasta_path = os.path.join(CALISMA_YOLU, raw_data)
    backup_fasta_path = os.path.join(output_dir, raw_data)

    if os.path.exists(original_fasta_path):
        shutil.copy2(original_fasta_path, backup_fasta_path)
    else:
        print(f"⚠️ UYARI: Orijinal fasta dosyası bulunamadı! ({raw_data})")

    print(f"📂 Hedef: {parts_dir}")
    print(f"📂 Çıktı: {output_dir}")
    print("-" * 60)

    recycle_count = cycle if 'cycle' in locals() else 3
    process_counter = 0

    for i in range(start_at, end_at + 1):
        process_counter += 1
        part_filename = f"{base_name_auto}_part{i}.fasta"

        input_file_path = f"{parts_dir}/{part_filename}"
        log_path = f"{output_dir}/{base_name_auto}_part{i}_log.txt"

        if os.path.exists(input_file_path):
            print(f"\n🔂 İŞLEM: [{process_counter}/{batch_quota}] | DOSYA: [{i}/{total_parts}] ('{part_filename}') Başlatılıyor...\n\n")

            current_config = {
                "raw_data": raw_data,
                "model_type": model_type_cli,
                "split_param": split_param if 'split_param' in locals() else "None",
                "cycle": recycle_count,
                "total_parts": total_parts,
                "processed_part": i,
                "kota": kota if 'kota' in locals() else "Hepsi",
                "take_up": take_up if 'take_up' in locals() else False,
                "resume_as": resume_as if 'resume_as' in locals() else 1
            }

            with open(log_path, "w") as logfile:
                json.dump(current_config, logfile)
                logfile.write("\n" + "="*50 + "\n")
                logfile.flush()

                command = [
                    "colabfold_batch",
                    input_file_path,
                    output_dir,
                    "--templates"
                ]

                if use_amber:
                    command.append("--amber")
                if use_gpu_relax:
                    command.append("--use-gpu-relax")

                command.extend([
                    "--num-recycle", str(recycle_count),
                    "--pair-mode", "unpaired_paired",
                    "--model-type", model_type_cli
                ])

                env = os.environ.copy()
                env["PYTHONUNBUFFERED"] = "1"

                process = subprocess.Popen(
                    command,
                    stdout=subprocess.PIPE,
                    stderr=subprocess.STDOUT,
                    text=True,
                    bufsize=1,
                    env=env
                )

                try:
                    for line in iter(process.stdout.readline, ''):
                        print(line, end='')
                        logfile.write(line)
                        logfile.flush()

                    process.wait()

                    if process.returncode == 0:
                        print(f"✅ Tamamlandı: {part_filename}")
                    else:
                        print(f"⚠️ Hata ile bitti: {part_filename} (Kod: {process.returncode})")

                except KeyboardInterrupt:
                    print(f"\n⛔ İŞLEM KULLANICI TARAFINDAN KESİLDİ! ({part_filename})\n")
                    print("🛑 Arka plan işlemleri fiziksel olarak durduruldu....")
                    process.kill()
                    process.wait()
                    sys.exit(1)

        else:
            print(f"⚠️ Dosya bulunamadı: {part_filename}")

    print(f"\n✅HEDEFLENEN ({process_counter}) ADET İŞLEM BİTTİ! Sonuçlar: '{output_dir}'")

except SystemExit:
    raise
except Exception as e:
    print(f"❌ BEKLENMEYEN HATA: {e}")
    raise SystemExit("Hata nedeniyle sonraki hücrelere geçiş durduruldu.")

🧹 Temizlik Yapıldı: '/content/drive/MyDrive/Folding/colabfold_results' ve içeriği silindi.
⚠️ UYARI: Orijinal fasta dosyası bulunamadı! (HPV.fasta)
📂 Hedef: /content/drive/MyDrive/Folding/HPV_fasta_parts
📂 Çıktı: /content/drive/MyDrive/Folding/colabfold_results
------------------------------------------------------------

🔂 İŞLEM: [1/1] | DOSYA: [1/121] ('HPV_part1.fasta') Başlatılıyor...


2026-02-02 15:42:13,090 Running colabfold 1.5.5 (83ee93d262a99ad62d6f0897c5ddd37eb918d385)

2026-02-02 15:42:42.358614: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1770046962.651898    1672 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1770046962.729533    1672 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for pl

### **KÖK DOSYASI TASNİFİ** - (optional)

**Kök dizini `colabfold_results` içinde karmaşık dosyaları derleyip düzenli bir tasnif yapılmasını sağlar.**

---
### ⚠️ UYARI :  **`RUN`** işlemi tamamlanmadan çalıştırmayınız.

---
####Not:
- Uzun süren `RUN` işleminden sonra oturumunuz sonlanabilir. Bu durumda **`Drive`** kök dizine bağlantı kurabilir önceki bölümleri tekrardan çalıştırmaya gerek kalmadan kök dizinde işlem yapılabilirsiniz.

- Kök dizinizin oluştuğu ve **RUN** işleminin  bittiğinden eminseniz aşağıdaki  `activing_code` kutucuğunu işaretleyerek devam edin.









In [8]:
import sys
import os
import shutil
import re
from google.colab import drive

activing_code = True # @param {"type":"boolean"}

if not activing_code:
    print("Çalıştırmak için kutucuğu işaretleyin.\n")
    sys.exit()

if not os.path.exists('/content/drive'):
    print("🔌 Google Drive bağlanıyor...")
    drive.mount('/content/drive')

base_path = "/content/drive/MyDrive/Folding/colabfold_results"

if not os.path.exists(base_path):
    print(f"❌ HATA: '{base_path}' yolu bulunamadı.")
    sys.exit()

for item in os.listdir(base_path):
    item_path = os.path.join(base_path, item)

    if os.path.isdir(item_path):
        sub_files = os.listdir(item_path)
        if "part" in item or "_env" in item or "_pairgreedy" in item:
             for sub_file in sub_files:
                src = os.path.join(item_path, sub_file)
                dst = os.path.join(base_path, sub_file)
                try:
                    shutil.move(src, dst)
                except: pass
             try:
                 shutil.rmtree(item_path)
             except: pass


files = [f for f in os.listdir(base_path) if os.path.isfile(os.path.join(base_path, f))]
moved_count = 0
pattern = re.compile(r"^(YP_\d+\.\d+|[A-Z0-9]+\.\d+|[A-Z0-9]+)")

for filename in files:
    if filename.endswith(".txt") or filename.endswith(".fasta") or filename.endswith(".json") or filename == "cite.bibtex" or filename.startswith("."):
        continue

    match = pattern.match(filename)
    if match:
        file_id = match.group(0)
        clean_id = file_id

        if clean_id == filename or len(clean_id) < 3:
            continue

        target_folder = os.path.join(base_path, clean_id)
        source_path = os.path.join(base_path, filename)
        dest_path = os.path.join(target_folder, filename)

        try:
            os.makedirs(target_folder, exist_ok=True)
            shutil.move(source_path, dest_path)
            moved_count += 1
        except Exception as e:
            print(f"⚠️ Taşıma hatası: {filename} -> {e}")

for item in os.listdir(base_path):
    item_path = os.path.join(base_path, item)
    if os.path.isdir(item_path):
        if item.endswith("_env") or item.endswith("_pairgreedy"):
            try: shutil.rmtree(item_path)
            except: pass
        elif not os.listdir(item_path):
            try: os.rmdir(item_path)
            except: pass

remaining_files = [f for f in os.listdir(base_path) if os.path.isfile(os.path.join(base_path, f))]
deleted_files = 0

for filename in remaining_files:
    if (filename.endswith(".txt") or
        filename.endswith(".fasta") or
        filename.endswith("config.json") or
        filename == "cite.bibtex" or
        filename.startswith(".")):
        continue

    file_path = os.path.join(base_path, filename)
    try:
        os.remove(file_path)
        deleted_files += 1
    except: pass

print(f"\n✅ İŞLEM TAMAMLANDI: {moved_count} dosya düzenlendi.")


✅ İŞLEM TAMAMLANDI: 9 dosya düzenlendi.


##### Tasnif işleminin doğruluğunu kontrolü için opsiyonel çalışır.

In [9]:
%cd  "/content/drive/MyDrive/Folding/colabfold_results"
!ls

/content/drive/MyDrive/Folding/colabfold_results
ALK24573.1	     cite.bibtex  HPV_part1_log.txt  templates_101
ALK24573.1.done.txt  config.json  log.txt


## SONUÇLARI KAYDETME
**Tüm sonuçları tek bir paket haline getirir ve bilgisayarınıza indirir**

In [10]:
import os
import shutil
import json
import glob
from google.colab import files

results_dir = "/content/drive/MyDrive/Folding/colabfold_results"

if os.path.exists(results_dir):
    for item in os.listdir(results_dir):
        item_path = f"{results_dir}/{item}"
        if os.path.isdir(item_path) and not os.listdir(item_path):
            try:
                os.rmdir(item_path)
            except: pass

if 'raw_data' not in locals():
    print("⚠️ Değişkenler hafızada yok. Log dosyasından okunuyor...")
    log_files = glob.glob(f"{results_dir}/*_part*_log.txt")

    if log_files:
        latest_log = max(log_files, key=os.path.getmtime)
        try:
            with open(latest_log, 'r') as f:
                first_line = f.readline().strip()
                if first_line.startswith("{") and first_line.endswith("}"):
                    config = json.load(f)

                    raw_data = config.get("raw_data", "unknown")
                    split_param = config.get("split_param", "None")
                    resume_as = config.get("resume_as", 1)
                    kota = config.get("kota", 1)
                    take_up = config.get("take_up", False)

                    print(f"✅ Config okundu: {raw_data}")
                else:
                    raise ValueError("Log formatı uymuyor.")
        except Exception as e:
            print(f"❌ Config hatası: {e}")
            raw_data = "result"; split_param = "None"; resume_as = 1; kota = 1; take_up = False
    else:
        print("❌ Log bulunamadı.")
        raw_data = "result"; split_param = "None"; resume_as = 1; kota = 1; take_up = False

try:
    clean_name = os.path.splitext(raw_data)[0]

    if take_up:
        end_val = int(resume_as) + int(kota) - 1
        zip_name = f"{clean_name}_part{resume_as}_to{end_val}_sp{split_param}"

    else:
        zip_name = f"{clean_name}_part{resume_as}_sp{split_param}"

except NameError:
    zip_name = "colabfold_results_backup"

output_zip_path = f"/content/{zip_name}"

if os.path.exists(results_dir):
    print(f"\n📦 '{zip_name}.zip' hazırlanıyor...")

    shutil.make_archive(output_zip_path, 'zip', results_dir)

    print("\n⬇️ İndirme başlatılıyor...")
    files.download(f"{output_zip_path}.zip")

    print(f"\n✅ İŞLEM TAMAM: '{zip_name}.zip' indirildi.")
else:
    print(f"❌ HATA: Sonuç klasörü yok: {results_dir}")


📦 'HPV_part6_sp1.zip' hazırlanıyor...

⬇️ İndirme başlatılıyor...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


✅ İŞLEM TAMAM: 'HPV_part6_sp1.zip' indirildi.
